# NY ACRIS and Property Valuation and Assessment Data Access

This file contains code to access the ACRIS and Property Valuation and Assessment Data tables from OpenData's Socrata API.

In [1]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import requests
import json
import os
import time
from tqdm import tqdm
from sodapy import Socrata

In [2]:
client = Socrata('data.cityofnewyork.us', None)

## 1. Property Valuation and Assessment (PVA) Data

This section accesses PVA data for all classes for a given BBL, this code can be modified to look up PVA data for multiple lots.

[PVA Table OpenData Link](https://data.cityofnewyork.us/City-Government/Property-Valuation-and-Assessment-Data-Tax-Classes/8y4t-faws) - *the data dictionary is also here*

Building Classes: https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html

Zoning Districts: https://www1.nyc.gov/site/planning/zoning/districts-tools.page

In [3]:
# Enter the BBL you want to lookup
borough = "3"
block = "2319"
lot = "8"

url = f"https://data.cityofnewyork.us/resource/8y4t-faws.json?boro='{borough}'&block='{block}'&lot='{lot}'"
result_rows = requests.get(url).json()
print(len(result_rows))
results_df = pd.DataFrame(result_rows)

6


In [4]:
# View some of the output, all available columns are listed in the table's data dictionary

# There are multiple results per lot for the past few tax years
results_df[["bldg_class", 'gross_sqft', 'lot_frt', 'bld_frt', 'lot_dep', 'bld_dep']]

,bldg_class,gross_sqft,lot_frt,bld_frt,lot_dep,bld_dep
0,S5,4600,23,23,80,50
1,S5,4600,23,23,80,50
2,S5,4600,23,23,80,50
3,S5,4600,23,23,80,50
4,S5,4600,23,23,80,50
5,S5,4600,23,23,80,50


In [5]:
# View more of the output, comment out what you want to ignore

results_df[[ #"parid", 
            "boro", "block", "lot",
            "year", "extracrdt",
            "bldg_class", "owner",
#             "zoning",
            "num_bldgs",
            "yrbuilt",'yralt1',
            'yralt1_range',
            #'yralt2', 'yralt2_range',
            "period", "land_area", "gross_sqft",
            #"hotel_area_gross", "office_area_gross",
            "residential_area_gross",
            "retail_area_gross",
            #"loft_area_gross", "factory_area_gross",
            #"warehouse_area_gross", 
            "garage_area",
#             "storage_area_gross",
            "other_area_gross",
            "bld_story", "units", "lot_frt", "lot_dep", "bld_dep"]].sort_values(by = "extracrdt", ascending = False)

,boro,block,lot,year,extracrdt,bldg_class,owner,num_bldgs,yrbuilt,yralt1,...,gross_sqft,residential_area_gross,retail_area_gross,garage_area,other_area_gross,bld_story,units,lot_frt,lot_dep,bld_dep
5,3,2319,8,2023,2022-01-10T00:00:00.000,S5,"2 ST. FELIX STREET A, LLC",1,1910,0,...,4600,4000,600,0,0,4,7,23,80,50
4,3,2319,8,2022,2021-05-24T00:00:00.000,S5,"2 ST. FELIX STREET A, LLC",1,1910,0,...,4600,4000,600,0,0,4,7,23,80,50
0,3,2319,8,2022,2021-01-11T00:00:00.000,S5,"VENESKI, BRIGITTE",1,1910,0,...,4600,4000,600,0,0,4,7,23,80,50
1,3,2319,8,2021,2020-05-17T00:00:00.000,S5,"VENESKI, BRIGITTE",1,1910,0,...,4600,4000,600,0,0,4,7,23,80,50
3,3,2319,8,2021,2020-01-10T00:00:00.000,S5,"VENESKI, BRIGITTE",1,1910,0,...,4600,0,0,0,0,4,7,23,80,50
2,3,2319,8,2020,2019-06-11T00:00:00.000,S5,"VENESKI, BRIGITTE",1,1910,0,...,4600,4000,600,0,0,4,7,23,80,50


In [6]:
# Obtain the address for the parcel

# list(results_df.columns)
borough_dictionary = {'1': 'Manhattan', '2': 'Bronx', '3': 'Brooklyn', '4': 'Queens', '5': 'Staten Island'}


housenum_lo = results_df['housenum_lo'].iloc[0].title()
housenum_hi = results_df['housenum_hi'].iloc[0].title()
if housenum_lo == housenum_hi:
    housenum = housenum_lo
if housenum_lo != housenum_hi:
    housenum = housenum_lo + "-" + housenum_hi

street_name = results_df['street_name'].iloc[0].title().replace(" ", "_")
boro = borough_dictionary[results_df['boro'].iloc[0]]

print(f'{housenum} {street_name.replace("_", " ")}, {boro}')

103 Berry Street, Brooklyn


In [7]:
# Save the data frame

results_df.to_csv(f'{housenum}_{street_name}_{boro}_pva.csv', index=False)

## 2. Primary Land Use Tax Lot Output (PLUTO) Table

Note: The 'boro' parameter is called 'borocode' for the pluto table. The tax class is in this table.

[Pluto Dictionary](https://data.cityofnewyork.us/City-Government/Primary-Land-Use-Tax-Lot-Output-PLUTO-/64uk-42ks)

In [8]:
url = f"https://data.cityofnewyork.us/resource/64uk-42ks.json?borocode='{borough}'&block='{block}'&lot='{lot}'"
result_rows = requests.get(url).json()
print(len(result_rows))
results_df = pd.DataFrame(result_rows)

1


In [9]:
# Calculates the Maximum FAR for the building and creates new coumns

try:
    results_df['current_building_area'] = float(results_df['lotarea']) * float(results_df['builtfar'])
except:
    results_df['current_building_area'] = 'builtfar only: ' + results_df['builtfar']

try:
    results_df['max_residential_area'] = float(results_df['lotarea']) * float(results_df['residfar'])
except:
    results_df['max_residential_area'] = 'residfar only: ' + results_df['residfar']

try:
    results_df['max_commercial_area'] = float(results_df['lotarea']) * float(results_df['commfar'])
except:
    results_df['max_commercial_area'] = 'commfar only: ' + results_df['commfar']

try:
    results_df['max_community_facility_area'] = float(results_df['lotarea']) * float(results_df['facilfar'])
except:
    results_df['max_community_facility_area'] = 'facilfar only: ' + results_df['facilfar']

In [10]:
results_df[['borough', 'block', 'lot',
            'lotarea', 'areasource', 'bldgarea',
            'unitstotal','unitsres', 'builtfar',
            #'comarea', 'resarea', 'officearea',
            #'retailarea', 'garagearea', 'strgearea',
            #'factryarea', 'otherarea', 'current_building_area',
            'residfar', 'max_residential_area', 'commfar',
            'max_commercial_area', 'facilfar', 'max_community_facility_area'
           ]]

,borough,block,lot,lotarea,areasource,bldgarea,unitstotal,unitsres,builtfar,residfar,max_residential_area,commfar,max_commercial_area,facilfar,max_community_facility_area
0,BK,2319,8,1840,2,4600,7,6,2.50000000000,2.00000000000,3680.0,0.00000000000,0.0,2.00000000000,3680.0


In [11]:
# Save the data frame

#  results_df.to_csv(f'{housenum}_{street_name}_{boro}_pluto.csv', index=False)